In [59]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d


import matplotlib.pyplot as plt
from scipy.stats import norm

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# 被験者ID
subject_id = "001"
# 実験番号
experiment_id = "001"

# IDT法のパラメータ
DISPERSION_THRESHOLD = 2  # deg
DURATION_THRESHOLD_MS = 100  # ms

eye_df = pd.read_csv(f"exported_csv/eye_df_id{subject_id}-{experiment_id}.csv")
sampling_df = pd.read_csv(f"exported_csv/sampling_df_id{subject_id}-{experiment_id}.csv")

In [60]:

# モニターサイズ(物理)
monitor_width_cm = 47.6
monitor_height_cm = 26.8
# モニター解像度(px)
monitor_resolution_px = (1920, 1080)
# 視距離(cm)
viewer_distance_cm = 60.0

# cm/pxの変換係数
cm_per_pixel_x = monitor_width_cm / monitor_resolution_px[0]
cm_per_pixel_y = monitor_height_cm / monitor_resolution_px[1]

# ディスプレイ中心を(0, 0)とするための変換
eye_df["gx_centered"] = eye_df["gx"] - 0.5
eye_df["gy_centered"] = eye_df["gy"] - 0.5

# 中心(0,0)での物理距離変換
eye_df["x_cm"] = eye_df["gx_centered"] * monitor_resolution_px[0] * cm_per_pixel_x
eye_df["y_cm"] = eye_df["gy_centered"] * monitor_resolution_px[1] * cm_per_pixel_y

# 視野角の計算
eye_df["x_deg"] = np.degrees(np.arctan2(eye_df["x_cm"], viewer_distance_cm))
eye_df["y_deg"] = np.degrees(np.arctan2(eye_df["y_cm"], viewer_distance_cm))

# データの有効性
eye_df["is_valid"] = eye_df["validity_sum"] > 1

print(eye_df.head())


         gx        gy     epoch_sec        hhmmss  validity_sum  trial  \
0       NaN       NaN  1.732521e+09  16:49:03.491             0     -1   
1  0.476050  0.816155  1.732521e+09  16:49:03.506             2     -1   
2  0.490610  0.818481  1.732521e+09  16:49:03.522             2     -1   
3  0.520527  0.754540  1.732521e+09  16:49:03.538             2     -1   
4  0.507679  0.780874  1.732521e+09  16:49:03.538             2     -1   

   gx_centered  gy_centered      x_cm      y_cm     x_deg     y_deg  is_valid  
0          NaN          NaN       NaN       NaN       NaN       NaN     False  
1    -0.023950     0.316155 -1.140020  8.472954 -1.088508  8.037926      True  
2    -0.009390     0.318481 -0.446964  8.535291 -0.426811  8.096281      True  
3     0.020527     0.254540  0.977085  6.821672  0.932965  6.486364      True  
4     0.007679     0.280874  0.365520  7.527423  0.349042  7.150799      True  


In [61]:
def interpolate_missing(df, time_col="epoch_sec", max_gap_ms=100):
    df= df.copy()
    df["valid"]= df["is_valid"]
    df["interp_x"]= np.nan
    df["interp_y"]= np.nan

    # 有効データを代入
    df.loc[df["valid"], "interp_x"] = df.loc[df["valid"], "x_deg"]
    df.loc[df["valid"], "interp_y"] = df.loc[df["valid"], "y_deg"]
    
    # 内部のみ線形補完
    df["interp_x"] = df["interp_x"].interpolate(limit_area="inside")
    df["interp_y"] = df["interp_y"].interpolate(limit_area="inside")
    
    # 無効区間の連続ブロックを取得
    invalid_mask = ~df["valid"]
    group_id = (invalid_mask != invalid_mask.shift()).cumsum()
    invalid_blocks = df[invalid_mask].groupby(group_id)

    for _, block in invalid_blocks:
        if len(block) == 0:
            continue
        t_start = block[time_col].iloc[0]
        t_end = block[time_col].iloc[-1]
        duration_ms = (t_end - t_start) * 1000
        if duration_ms > max_gap_ms:
            # 100ms超えたら補完結果をNaNに戻す
            df.loc[block.index, ["interp_x", "interp_y"]] = np.nan
    

    return df
    
    

In [62]:
def apply_gaussian_filter_by_block(df, col_x="x_deg", col_y="y_deg", sigma=1.0):
    df = df.copy()
    df["valid"]= df["is_valid"]
    df["filtered_x"] = np.nan
    df["filtered_y"] = np.nan

    # 有効なデータ（NaNでない）だけを連続ブロックとして抽出
    valid_mask = df[col_x].notna() & df[col_y].notna()
    block_id = (valid_mask != valid_mask.shift()).cumsum()
    blocks = df[valid_mask].groupby(block_id)

    for _, block in blocks:
        idx = block.index
        smoothed_x = gaussian_filter1d(block[col_x], sigma=sigma)
        smoothed_y = gaussian_filter1d(block[col_y], sigma=sigma)
        df.loc[idx, "filtered_x"] = smoothed_x
        df.loc[idx, "filtered_y"] = smoothed_y

    return df


In [ ]:
def detect_fixations_idt(df, dispersion_threshold=DISPERSION_THRESHOLD , duration_threshold_ms=DURATION_THRESHOLD_MS):
    fixations = []
    timestamps = df["epoch_sec"].to_numpy()
    xs = df["filtered_x"].to_numpy()
    ys = df["filtered_y"].to_numpy()
    
    i=0
    while i < len(df):
        if np.isnan(xs[i]) or np.isnan(ys[i]):
            i += 1
            continue
        
        window = [(xs[i], ys[i])]
        t_start = timestamps[i]
        j = i + 1
        
        while j < len(df):
            if np.isnan(xs[j]) or np.isnan(ys[j]):
                break
            window.append((xs[j], ys[j]))
            t_end = timestamps[j]
            duration = (t_end - t_start) * 1000
            
            x_vals, y_vals = zip(*window)
            x_center = np.mean(x_vals)
            y_center = np.mean(y_vals)
            
            distances = np.sqrt((np.array(x_vals) - x_center) ** 2 + (np.array(y_vals) - y_center) ** 2)
            
            if np.max(distances) > dispersion_threshold:
                break
            if duration >= duration_threshold_ms:
                fixations.append({
                    "start_time": t_start,
                    "end_time": t_end,
                    "duration_ms": duration,
                    "x_mean_deg": np.mean(x_vals),
                    "y_mean_deg": np.mean(y_vals),
                })
            j += 1
        i = j
    return pd.DataFrame(fixations)
    

In [64]:
new1_df=interpolate_missing(eye_df)
print(new1_df.head())
new2_df=apply_gaussian_filter_by_block(new1_df)
detect_fixations_idt(new2_df)


         gx        gy     epoch_sec        hhmmss  validity_sum  trial  \
0       NaN       NaN  1.732521e+09  16:49:03.491             0     -1   
1  0.476050  0.816155  1.732521e+09  16:49:03.506             2     -1   
2  0.490610  0.818481  1.732521e+09  16:49:03.522             2     -1   
3  0.520527  0.754540  1.732521e+09  16:49:03.538             2     -1   
4  0.507679  0.780874  1.732521e+09  16:49:03.538             2     -1   

   gx_centered  gy_centered      x_cm      y_cm     x_deg     y_deg  is_valid  \
0          NaN          NaN       NaN       NaN       NaN       NaN     False   
1    -0.023950     0.316155 -1.140020  8.472954 -1.088508  8.037926      True   
2    -0.009390     0.318481 -0.446964  8.535291 -0.426811  8.096281      True   
3     0.020527     0.254540  0.977085  6.821672  0.932965  6.486364      True   
4     0.007679     0.280874  0.365520  7.527423  0.349042  7.150799      True   

   valid  interp_x  interp_y  
0  False       NaN       NaN  
1   Tr

,start_time,end_time,duration_ms,x_mean_deg,y_mean_deg
0,1.732521e+09,1.732521e+09,108.999968,1.938270,8.513012
1,1.732521e+09,1.732521e+09,125.000000,1.973824,8.463294
2,1.732521e+09,1.732521e+09,141.000032,1.987016,8.396333
3,1.732521e+09,1.732521e+09,155.999899,1.974493,8.324779
4,1.732521e+09,1.732521e+09,171.999931,1.941901,8.272222
...,...,...,...,...,...
17412,1.732522e+09,1.732522e+09,126.999855,-11.524944,15.211944
17413,1.732522e+09,1.732522e+09,142.999887,-11.526589,15.160022
17414,1.732522e+09,1.732522e+09,158.999920,-11.526968,15.096107
17415,1.732522e+09,1.732522e+09,109.999895,-10.909634,16.292982
